In [1]:
import tensorflow as tf
import numpy as np

In [2]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/home/guhangsong/Data/aclImdb/train",
    batch_size = batch_size,
    validation_split = 0.2,
    subset = "training",
    seed = 1337
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/home/guhangsong/Data/aclImdb/train",
    batch_size = batch_size,
    validation_split = 0.2,
    subset = "validation",
    seed = 1337
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/home/guhangsong/Data/aclImdb/test",
    batch_size = batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-03-16 20:59:14.229306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-16 20:59:14.288254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-16 20:59:14.288622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-16 20:59:14.290091: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow wi

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [4]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [5]:
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")

max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = tf.keras.layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = max_features,
    output_mode = "int",
    output_sequence_length = sequence_length
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [6]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size = 10)
val_ds = val_ds.cache().prefetch(buffer_size = 10)
test_ds = test_ds.cache().prefetch(buffer_size = 10)

In [7]:
inputs = tf.keras.Input(shape = (None,), dtype=tf.int64)

x = tf.keras.layers.Embedding(max_features, embedding_dim)(inputs)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(128, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

predictions = tf.keras.layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])


In [8]:
epochs = 3

model.fit(train_ds, validation_data = val_ds, epochs=epochs)

Epoch 1/3


2022-03-16 21:24:23.088242: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-03-16 21:24:25.907251: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


625/625 [==============================] - 9s 7ms/step - loss: 0.4137 - accuracy: 0.7890 - val_loss: 0.2699 - val_accuracy: 0.8898
Epoch 2/3
625/625 [==============================] - 4s 6ms/step - loss: 0.1694 - accuracy: 0.9358 - val_loss: 0.2908 - val_accuracy: 0.8858
Epoch 3/3
625/625 [==============================] - 4s 7ms/step - loss: 0.0875 - accuracy: 0.9697 - val_loss: 0.3821 - val_accuracy: 0.8836


In [9]:
model.evaluate(test_ds)

782/782 [==============================] - 2s 2ms/step - loss: 0.4262 - accuracy: 0.8723


[0.4261969029903412, 0.8723199963569641]

In [10]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)

indices = vectorize_layer(inputs)

outputs = model(indices)

end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"]
)

end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 3s 4ms/step - loss: 0.4262 - accuracy: 0.8723


[0.42619720101356506, 0.8723199963569641]